## Work
1. 請嘗試將 preproc_x 替換成以每筆資料的 min/max 進行標準化至 -1 ~ 1 間，再進行訓練
2. 請嘗試將 mlp 疊更深 (e.g 5~10 層)，進行訓練後觀察 learning curve 的走勢
3. (optional) 請改用 GPU 進行訓練 (如果你有 GPU 的話)，比較使用 CPU 與 GPU 的訓練速度

In [3]:
## 
"""
Your code here (optional)
確認硬體資源
"""
!nvidia-smi

'nvidia-smi.exe' 不是內部或外部命令、可執行的程式或批次檔。


In [13]:
import os
import keras
from keras.utils import multi_gpu_model

# 請嘗試設定 GPU：os.environ
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']="0"

In [14]:
train, test = keras.datasets.cifar10.load_data()

In [15]:
## 資料前處理
"""
Your code here
"""
def preproc_x(x, flatten=True):
    x = x/255
    if flatten:
        x = x.reshape(len(x),-1)
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1]==1:
        y = keras.utils.to_categorical(y, num_classes)
    return y

In [16]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [17]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(50000, 3072)
(10000, 3072)
(50000, 10)
(10000, 10)


In [18]:
"""Code Here
建立你的神經網路
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i==0:
            x = keras.layers.Dense(units=n_units, activation='relu', name='hidden_layer'+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation='relu', name='hidden_layer'+str(i+1))(x)
            
    out = keras.layers.Dense(units=output_units, activation='softmax',name='output')(x)
    
    model = keras.models.Model(inputs=[input_layer],outputs =[out])
    return model

In [19]:
model = build_mlp(input_shape=x_train.shape[1:])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________


In [20]:
## 超參數設定
LEARNING_RATE = 0.001
EPOCHS = 100
BATCH_SIZE = 256

In [21]:
optimizer = keras.optimizers.Adam(lr=LEARNING_RATE)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

In [22]:
model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 10s 203us/step - loss: 1.9408 - accuracy: 0.2989 - val_loss: 1.7934 - val_accuracy: 0.3530
Epoch 2/100
50000/50000 [==============================] - 10s 202us/step - loss: 1.7084 - accuracy: 0.3872 - val_loss: 1.6787 - val_accuracy: 0.4101
Epoch 3/100
50000/50000 [==============================] - 10s 195us/step - loss: 1.6202 - accuracy: 0.4184 - val_loss: 1.6357 - val_accuracy: 0.4201
Epoch 4/100
50000/50000 [==============================] - 10s 196us/step - loss: 1.5500 - accuracy: 0.4462 - val_loss: 1.5186 - val_accuracy: 0.4562
Epoch 5/100
50000/50000 [==============================] - 10s 193us/step - loss: 1.5022 - accuracy: 0.4614 - val_loss: 1.5039 - val_accuracy: 0.4611
Epoch 6/100
50000/50000 [==============================] - 10s 201us/step - loss: 1.4738 - accuracy: 0.4754 - val_loss: 1.4807 - val_accuracy: 0.4680 accura
Epoch 7/100
50000/50000 [==================

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt


train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]

train_acc = model.history.history["acc"]
valid_acc = model.history.history["val_acc"]

plt.plot(range(len(train_loss)), train_loss, label="train loss")
plt.plot(range(len(valid_loss)), valid_loss, label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc)), train_acc, label="train accuracy")
plt.plot(range(len(valid_acc)), valid_acc, label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()